In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from importlib import reload
import extract_features
import os
import numpy as np
import json
import seaborn as sns
import results_functions
from scipy import stats
import pickle
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests


json_path_project = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\BATTERY\\'
json_path_onedrive = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\'

### Make descriptive boxplots for all

In [36]:
directory = os.path.join(json_path_onedrive,
    'results', 'Avg_Features', 'Avg_Features_Tbls' )

dir_saving = os.path.join(json_path_onedrive,
    'results', 'Avg_Features', 'test_results' )
reload(results_functions)
%matplotlib qt
saving = 1
df_fu0m, df_fu3m, df_fu12m, all_dfs = results_functions.get_descriptives(directory, dir_saving, saving)

'''
with open(os.path.join(directory,
    'Means_FU0M.pkl'), "rb") as file:
    val_dat = pickle.load(file)
'''

t:\Dokumente\PROJECTS\BATTERY_LIFE\PerceptBatteryLife\PerceptIPGProgression\results_functions.py:58: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  means = this_df.mean()
t:\Dokumente\PROJECTS\BATTERY_LIFE\PerceptBatteryLife\PerceptIPGProgression\results_functions.py:59: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  stds = this_df.std()


'\nwith open(os.path.join(directory,\n    \'Means_FU0M.pkl\'), "rb") as file:\n    val_dat = pickle.load(file)\n'

In [13]:
print(np.round(np.mean(df_fu12m['AccTimeSinceImplant_Days']), decimals = 2))
print(np.round(np.std(df_fu12m['AccTimeSinceImplant_Days']), decimals = 2))


376.89
40.03


In [22]:
#Make Battery boxplot
all_dfs.head()
fig, axs = plt.subplots(1, 1, figsize=(5,5))
sns.boxplot(data=all_dfs, x="TimePoint", y='FirstBatVal',fliersize=0, 
            boxprops=dict(facecolor='pink', edgecolor='darkred', linewidth = 3),
            whiskerprops=dict(color='darkred', linewidth = 4), width = 0.5, dodge = 0.2)

sns.stripplot(data=all_dfs, x='TimePoint', y='FirstBatVal', marker='o',
                jitter = True, size = 12, alpha = 0.4, color = 'firebrick')
axs.set_ylabel('IPG Battery [%]')
plt.savefig(os.path.join(dir_saving, 'BatteryPercentageTP'), dpi = 200)
plt.savefig(os.path.join(dir_saving, 'BatteryPercentageTP.pdf'))

In [4]:
all_dfs.head()

,SubID,TimePoint,Telemetry_AllSec,TelemDurSumSecRes,TelemDurSumSecWard,SensDurSumSec,FirstBatVal,LastBatVal,AccTimeSinceImplant_Days,Electrode,Telemetry_AllMin,TelemDurSumSMinRes,TelemDurSumMinWard,SensDurSumMin
0,Sub002,FU0M,2412.108,2184.108,228.0,584.716,99.0,99.0,0.0,3389,40.201800,36.401800,3.800000,9.745267
0,Sub005,FU0M,3005.000,2062.000,128.0,821.772,99.0,99.0,0.0,3389,50.083333,34.366667,2.133333,13.696200
0,Sub006,FU0M,331.000,0.000,331.0,0.000,99.0,99.0,0.0,3389,5.516667,0.000000,5.516667,0.000000
0,Sub007,FU0M,1876.000,0.000,1451.0,254.356,99.0,99.0,0.0,3389,31.266667,0.000000,24.183333,4.239267
0,Sub008,FU0M,171.000,0.000,171.0,0.000,99.0,99.0,0.0,3389,2.850000,0.000000,2.850000,0.000000


#### Pairwise Comparisons

In [ ]:
all_fus_df = pd.read_excel(os.path.join(
    json_path_onedrive, 'results', 'Avg_Features', 'test_results','All_FollowUp_dfs.xlsx'
)) 

all_fus_df.head()

In [ ]:
# Create a dictionary to store the results for each column
all_columns = ['Telemetry_AllMin', 'SensDurSumMin']
wilcoxon_results = {}

# List of time points to compare
time_points = ['FU0M', 'FU3M', 'FU12M']

# Perform pairwise Wilcoxon signed-rank tests and store the results for each column
for column in all_columns:
    comparisons_results = {}
    for i in range(len(time_points)-1):
        for j in range(i+1, len(time_points)):
            tp1, tp2 = time_points[i], time_points[j]
            x1 = all_fus_df.loc[all_fus_df['TimePoint'] == tp1,column]#all_fus_df.loc[all_fus_df['TimePoint'] == tp1, column]
            x2 = all_fus_df.loc[all_fus_df['TimePoint'] ==  tp2, column]#all_fus_df.loc[all_fus_df['TimePoint'] == tp2, column]

            statistic, p_value = wilcoxon(x1, x2, nan_policy='omit')

            comparison_name = f"{column}_{tp1}-{tp2}"


            comparisons_results[comparison_name] = {'Statistic': statistic, 'Original_p-values': p_value}
    
    wilcoxon_results[column] = comparisons_results

# Convert the dictionary to a DataFrame
results_df = pd.DataFrame({(column, key): value for column, values in wilcoxon_results.items() for key, value in values.items()}).T


## Adjust for multiple comparisons
reject, corrected_p_values, _, _ = multipletests(results_df['Original_p-values'],
                                                 alpha = 0.05, 
                                                 method='bonferroni')
results_df['Corrected_p-values'] = corrected_p_values

significance_conds = [
    (results_df['Corrected_p-values'] <= 0.001),
    (results_df['Corrected_p-values'] <= 0.01),
    (results_df['Corrected_p-values'] < 0.05),
    (results_df['Corrected_p-values'] >= 0.05)
]

values = ['***', '**', '*', 'n.s.']

results_df['Significance_multcomp'] = np.select(significance_conds, values, default = 'Other')
results_df

In [ ]:
results_df.to_excel(os.path.join(
    json_path_onedrive, 'results', 'Avg_Features', 'test_results','PairwiseComps.xlsx'
))

### Make correlations with TEED

In [ ]:
directory_Feat = os.path.join(json_path_onedrive,
    'results', 'Avg_Features', 'Avg_Features_Tbls')

directory_TEED = os.path.join(json_path_onedrive,
    'results', 'Stim_pars', 'TEED')

directory_corrs = os.path.join(json_path_onedrive,
    'results', 'Correlations')

saving = 0

In [ ]:
reload(results_functions)
%matplotlib qt
corr_df  = results_functions.get_battery_corr_df(directory_Feat, 
                                                directory_TEED, 
                                                directory_corrs, 
                                                saving)



## Multivariate Analysis

In [ ]:
#Identify and remove Outlierts with the IQR Method
corr_df = pd.read_csv(os.path.join(json_path_onedrive, 'results','MultivariateAnal',
                                   'Corr_df.csv'), index_col=None)
corr_df.head()

IQR Method (Interquartile Range):

It uses the range between the first quartile (Q1) and the third quartile (Q3) to identify outliers.

Data points outside the range [Q1 - 1.5 * IQR, Q3 + 1.5 * IQR] are considered outliers.

In [ ]:
values_of_int = ['Telemetry_AllSec_div', 'SensDurSumSec_div', 'Chronic_12mfu_Days', 'TEED']

for val in values_of_int:
    data = corr_df[val]
    
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    outliers = (data < q1 - 1.5 * iqr) | (data > q3 + 1.5 * iqr)
    outlier_subids = corr_df.loc[outliers, 'SubID'].tolist()
    
    print(f'Outliers for {val} is {tuple(outlier_subids)}')
    

In [ ]:
directory_corrs = os.path.join(json_path_onedrive,
    'results', 'Correlations')

reload(results_functions)
saving = 0
filtered_corr_df = corr_df[~corr_df['SubID'].isin(['Sub015','Sub021', 'Sub029', 'Sub030'])]
correlation_stats = results_functions.corrs_scatters(filtered_corr_df, saving, directory_corrs)
#correlation_stats

In [ ]:
correlation_stats

In [ ]:
#1. Correct the spearman correlations for multiple comparisons

corrs_pvalues = [item['p-value'] for item in list(correlation_stats.values())]

reject, corrected_p_values, _, _ = multipletests(corrs_pvalues,
                                                 alpha = 0.05, 
                                                 method='bonferroni')

corrected_p_values

## Multivariate Analysis

In [ ]:
#Multiple Linear Regression
import statsmodels.api as sm

# Specify the formula for the mixed-effects model
filtered_corr_df[['Telemetry_AllSec_div', 'SensDurSumSec_div', 'Chronic_12mfu_Days', 'TEED']] = filtered_corr_df[['Telemetry_AllSec_div', 'SensDurSumSec_div', 'Chronic_12mfu_Days', 'TEED']].astype(float)
X = filtered_corr_df[['Telemetry_AllSec_div', 'SensDurSumSec_div', 'Chronic_12mfu_Days', 'TEED']]
X = sm.add_constant(X)  # add a constant term for the intercept
y = filtered_corr_df['Battery_12mfu']

model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
#coefficients/Parameter estimates
model.params
#const = intercept
#coefficient for TEED


In [ ]:
%matplotlib qt
coefficients = model.params

xlabels = ['Total Telemetry Duration [min]',
           'Total Active Sensing Duration [min]',
           'Total Chronic Sensing Duration [days]',
           'TEED [Joules/sec]']

# Scatterplots with regression lines
fig, axes = plt.subplots(2, 2, figsize=(8, 8))

for i, var in enumerate(X.columns[1:]):  # Exclude the constant term
    row, col = divmod(i, 2)
    ax = axes[row, col]

    # Scatterplot
    ax.scatter(X[var], y, alpha=0.5, s = 80)

    # Regression line
    m, b = np.polyfit(X[var], y, 1)
    ax.plot(X[var], m*X[var]+b, linewidth = 2)
    
    ax.set_xlabel(xlabels[i])
    ax.set_ylabel('IPG Battery [%]')
    #ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.savefig(os.path.join(json_path_onedrive, 'results','MultivariateAnal', 'MultiVarScatters'), dpi = 200)

In [ ]:
x = filtered_corr_df['Telemetry_AllSec_div']
y = filtered_corr_df['TEED']

plt.scatter(x,y)
stats = stats.spearmanr(x,y, nan_policy = 'omit')
stats

#### Test assumptions for linear regression

In [ ]:
#Test Assumptions:
import seaborn as sns

#1. Linearity and 
residuals = model2.resid
fitted_values = model2.fittedvalues

fig, axs = plt.subplots(1,2)
axs[0].scatter(fitted_values, residuals)
axs[0].set_xlabel('Fitted Values')
axs[0].set_ylabel('Residuals')
axs[0].set_title('Residuals vs. Fitted Values Plot')

#Interpretation: Check for a random scatter of points with no discernible pattern. 
# A pattern may indicate non-linearity or heteroscedasticity.

#2. Homoscedasticity
axs[1].scatter(fitted_values, abs(np.sqrt(np.abs(residuals))))
axs[1].set_xlabel('Fitted Values')
axs[1].set_ylabel('Square Root of Standardized Residuals')
axs[1].set_title('Scale-Location Plot')
plt.show()
#Interpretation: Check for a horizontal line with no clear pattern. 
# A funnel-shaped pattern may indicate heteroscedasticity.

In [ ]:
#3. Normality of Residuals
sm.qqplot(residuals, line='s')
#Interpretation: Points close to the diagonal 
# line suggest that residuals are approximately normally distributed.

from scipy.stats import shapiro

stat, p_value = shapiro(residuals)

print(f'Shapiro-Wilk Test Statistic: {stat:.4f}, p-value: {p_value:.4f}')
#small p-value suggests that the residuals are not normally distributed.